In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import earthpy as et
import glob
import os
import pickle # Loads and saves ML models
from shapely.geometry import mapping
from rasterio.mask import mask
import earthpy.mask as em
from skimage import exposure
from rasterio.plot import show
import math
from xml.dom import minidom

In [2]:
shapefile_path =  glob.glob('/home/emmanuel/Desktop/BIg data Summer schoo/Gaulunia_2017/GauluniaD_P1/*shp')
rasters_path = '/media/emmanuel/Transcend/SOYBEAN PROJECT/DATA COLLECTION/Gaulunia2017'
try:
    os.remove(shapefile_path[0][:-4] + 'NDVI.txt')
    os.remove(shapefile_path[0][:-4] + 'EVI.txt')
except:
    pass

shapefile_path[0][:-3] + 'txt'
shapefile_path[0][70:-4]
shapefile_path[0]
shapefile_path[0][:-4] + 'EVI.txt'
#/home/emmanuel/Desktop/BIg data Summer schoo/Syngenta_2018/Syngentap4/SyngentaP4EVI.txt

'/home/emmanuel/Desktop/BIg data Summer schoo/Gaulunia_2017/GauluniaD_P1/GauluniaD_P1EVI.txt'

In [3]:
shapefile = gpd.read_file(shapefile_path[0])


In [4]:
def IndexCalc(array1, array2):
    np.seterr(divide = 'ignore', invalid = 'ignore')
    Index = (array1 - array2)/(array1 + array2)
    return Index

In [6]:
def IndexEVI(array1, array2, array3):
    np.seterr(divide = 'ignore', invalid = 'ignore')
    EVI = 2.5 * ((array1 - array2) / (array1 + 6 * array2 - 7.5 * array3 + 1))
    return EVI

In [7]:
def Final_Array_Cleaner(raster_array):
    better_array  = np.where((raster_array <= 1) & (raster_array > 0), raster_array, np.nan)
    return better_array

In [8]:
def Array_Cleaner(raster_array):
    better_array  = np.where((raster_array <= 1) & (raster_array >= 0), raster_array, 0.0)
    return better_array

In [9]:
def Index_Cleaneer(index_array):
    Index_array = np.where((index_array <= 1) & (index_array >= -1), index_array, 0.0)
    return Index_array

In [10]:
PlanetscopeClassifier = pickle.load(open('PlanetScopeClassifier.sav', 'rb'))
Landsat8Classifier = pickle.load(open('Landsat8_Classifier.sav', 'rb'))
Sentinel2Classifier = pickle.load(open('Sentinel2Classifier.sav', 'rb'))

In [11]:
def getFeatures(gdf):
    '''Function to parse features from GeoDataFrame in such a manner that rasterio can take them'''
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [12]:
def allPixels(blue, green, red, nir, NDVI, NDWI):
    all_df = pd.DataFrame()
    df_ls = []
    
    df_ls.append(pd.DataFrame({'Blue_Band':blue.read(1).ravel(),
                              'Green_Band':green.read(1).ravel(),
                              'Red_Band':red.read(1).ravel(),
                              'NIR_Band':nir.read(1).ravel(),
                              'NDVI_Band':NDVI.read(1).ravel(),
                              'NDWI_Band':NDWI.read(1).ravel()}))
    all_df = pd.concat(df_ls, axis = 0)
    return all_df

In [13]:
def RefCoeffs(filename):
    xmldoc = minidom.parse(filename[:-4] + '_metadata.xml')
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")
    filename[:-4] + '_metadata.xml'
    
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
    return coeffs

In [14]:
def allPixels1(blue, green, red, nir, NDVI, NDWI):
    blue = Array_Cleaner(blue)
    green = Array_Cleaner(green)
    red = Array_Cleaner(red)
    nir = Array_Cleaner(nir)
    NDVI =  Index_Cleaneer(NDVI)
    NDWI =  Index_Cleaneer(NDWI)
    all_df = pd.DataFrame()
    df_ls = []
    
    df_ls.append(pd.DataFrame({'Blue_Band':blue.ravel(),
                              'Green_Band':green.ravel(),
                              'Red_Band':red.ravel(),
                              'NIR_Band':nir.ravel(),
                              'NDVI_Band':NDVI.ravel(),
                              'NDWI_Band':NDWI.ravel()}))
    all_df = pd.concat(df_ls, axis = 0)
    return all_df

In [15]:
def CropRaster(shapefile, raster):
    raster= rasterio.open(raster)
    shapefile = gpd.read_file(shapefile)
    shapefile = shapefile.to_crs(crs = raster.crs.data)
    extent_geojson = mapping(shapefile['geometry'][0])

    raster_crop, crop_affine = mask(raster, shapes = [extent_geojson], crop = True)
    raster_meta = raster.meta.copy()
    raster_meta.update({'driver': 'GTiff', 'height':raster_crop.shape[1],
                        'width':raster_crop.shape[2], 'transform':crop_affine})
    cr_ext = rasterio.transform.array_bounds(raster_meta['height'],
                                             raster_meta['width'], raster_meta['transform'])
    bound_order = [0]
    cr_extent = [cr_ext[b] for b in bound_order]
    cropedraster = raster_crop[0] 
   
    return cropedraster

In [16]:
def WriteTofile(date, shapefile_path, Ave_NDVI, Ave_EVI):
    month = date[4:6]
    date = date[6:8]
    if int(month) == 11:
        date = int(date) - 9
    elif int(month) == 12:
        date = 11 + int(date)
    elif int(month) == 1:
        date = 42 + int(date)
    elif int(month) == 2:
        date = 73 + int(date)
    elif int(month) == 3:
        date = 101 + int(date)
    elif int(month) == 4:
        date = 132 + int(date)
    elif int(month) == 5:
        date = 162 + int(date)
    datafile = shapefile_path[0][:-4] + 'NDVI.txt'
    datafile2 = shapefile_path[0][:-4] + 'EVI.txt'
    f = open(datafile, 'a+')
    f2 = open(datafile2, 'a+')
    yes = os.stat(datafile).st_size  == 0
    yes = os.stat(datafile2).st_size == 0
    if yes is False:
        f.write(str(date) + '        ' + str(Ave_NDVI) + '\n')
        f2.write(str(date) + '        ' + str(Ave_EVI) + '\n')
    else:
        f.write('Date           Average_NDVI\n')
        f2.write('Date          Average_EVI\n')
        f.write(str(date) + '        ' + str(Ave_NDVI) + '\n')
        f2.write(str(date) + '        ' + str(Ave_EVI) + '\n')
    f.close()

In [31]:
# Analysing Sentinel 2 Rasters
dateblue = 4
dategreen = 90
datered = 76
datenir = 92
print('start')
for subdir, dirs, files in os.walk(rasters_path):  # For loop that finds rasters according to the band
    for file in files:
        #try:
        if file.endswith('B02.tif'): #
            dateblue = file[10:18]
            blue = os.path.join(subdir,file)
        if file.endswith('B03.tif'):
            dategreen = file[10:18]
            green = os.path.join(subdir,file)
        if file.endswith('B04.tif'):
            datered = file[10:18]
            red = os.path.join(subdir, file)
        if file.endswith('B08.tif'):
            datenir = file[10:18] 
            nir = os.path.join(subdir, file)
        if dateblue == dategreen  and dategreen == datered and datered == datenir:
            try:
                blue_crop =  CropRaster(shapefile_path[0], blue)
                green_crop =  CropRaster(shapefile_path[0], green)
                red_crop =  CropRaster(shapefile_path[0], red)
                nir_crop =  CropRaster(shapefile_path[0], nir)


                NDVI_crop = IndexCalc(nir_crop, red_crop)
                NDWI_crop = IndexCalc(green_crop, nir_crop)
                if np.nanmean(blue_crop) != 0.0 and np.nanmean(NDVI_crop) != 0.0:
                    #print(dateblue, dategreen, datered, datenir)
                    rgb = np.dstack((red_crop, green_crop, blue_crop))
                    better_rgb = exposure.rescale_intensity(rgb, in_range=(0, 0.25))
                    cdf = allPixels1(blue_crop, green_crop, red_crop, nir_crop, NDVI_crop, NDWI_crop)

                    img = cdf.iloc[:,:].values
                    pred_class = Sentinel2Classifier.predict(img)
                    landclass = pred_class.reshape(blue_crop.shape)
                    landclass = np.array(landclass, dtype = float)
                    l_class3 = landclass == 3
                    l_class3 = l_class3.astype(np.float)
                    nir_crop = nir_crop*l_class3
                    red_crop = red_crop*l_class3

                    nir_crop1 = Final_Array_Cleaner(nir_crop)
                    red_crop1 = Final_Array_Cleaner(red_crop)
                    blue_crop1 = Final_Array_Cleaner(blue_crop)
                    NDVIcrop1 = IndexCalc(nir_crop1, red_crop1)
                    EVIcrop1 = IndexEVI(nir_crop1, red_crop1, blue_crop1)
                    Ave_NDVI = np.nanmean(NDVIcrop1)
                    Ave_EVI = np.nanmean(EVIcrop1)
                    
#                     fig = plt.figure(figsize = (15, 15))
#                     plt.imshow(better_rgb)
#                     plt.savefig(shapefile_path[0][:-4]+ '_' + dateblue +  'T.png')
#                     plt.close(fig)
                
                    if not math.isnan(Ave_NDVI):
                        WriteTofile(dateblue, shapefile_path, Ave_NDVI, Ave_EVI)
                        print(dateblue, Ave_NDVI)


    #                     classes = ['Clouds','Shadows','Data']
    #                     length = len(classes)
    #                     fig = plt.figure(figsize=(15,15))
    #                     cmap = plt.cm.get_cmap("Set1", length)
    #                     plt.imshow(landclass, cmap=cmap)
    #                     ticks = np.arange(1, length + 1)
    #                     colorbar = plt.colorbar(ticks = ticks, orientation="horizontal")
    #                     colorbar.set_ticklabels(classes)
    #                     plt.savefig('Syngenta_2018/ClassifiedImage.png')
    #                     plt.close(fig )

                        fig = plt.figure(figsize = (20, 15))
                        plt.rc('xtick', labelsize = 20)
                        plt.rc('ytick', labelsize = 20)
                        plt.subplot(221)
                        plt.title('Sentinel NDVI\n', fontsize = 20 , fontweight = 'bold')
                        plt.imshow(NDVIcrop1, cmap = plt.cm.nipy_spectral)
                        plt.subplot(222)
                        plt.title('True Color Image\n', fontsize = 20, fontweight = 'bold')
                        plt.imshow(better_rgb)
                        plt.savefig(shapefile_path[0][:-4]+ '_' + dateblue +  '.png')
                        plt.close(fig)
            except:
                pass
            #show(blue_crop)

            dateblue = 8
            dategreen = 6
            datered = 98
            datenir = 90

start


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice


20170125 0.6845197284380944
20170214 0.8764422194919583


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice


20170306 0.6457136774568485
20170316 0.8279411887682886
20170326 0.47200317451041884


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice


20170505 0.22528326954239017


In [32]:
# Analysing Landsat 8 rasters
dateblue = 4
dategreen = 90
datered = 76
datenir = 92

for subdir, dirs, files in os.walk(rasters_path):  # For loop that finds rasters according to the band
    for file in files:
        if file.endswith('band2.tif'): #
            dateblue =  file[17:25]
            blue = os.path.join(subdir,file)
    
        if file.endswith('band3.tif'):

            dategreen = file[17:25]
            green = os.path.join(subdir,file)
        if file.endswith('band4.tif'):
            datered = file[17:25]
            red = os.path.join(subdir, file)
        if file.endswith('band5.tif'):
            datenir = file[17:25] 
            nir = os.path.join(subdir, file)
        if dateblue == dategreen  and dategreen == datered and datered == datenir:
            try:
                blue_crop =  CropRaster(shapefile_path[0], blue)*0.0001
                green_crop =  CropRaster(shapefile_path[0], green)*0.0001
                red_crop =  CropRaster(shapefile_path[0], red)*0.0001
                nir_crop =  CropRaster(shapefile_path[0], nir)*0.0001


                NDVI_crop = IndexCalc(nir_crop, red_crop)
                NDWI_crop = IndexCalc(green_crop, nir_crop)
                if np.nanmean(blue_crop) != 0.0 and np.nanmean(NDVI_crop) != 0.0:
                    rgb = np.dstack((red_crop, green_crop, blue_crop))
                    better_rgb = exposure.rescale_intensity(rgb, in_range=(0, 0.25))
                    cdf = allPixels1(blue_crop, green_crop, red_crop, nir_crop, NDVI_crop, NDWI_crop)

                    img = cdf.iloc[:,:].values
                    pred_class = Landsat8Classifier.predict(img)
                    landclass = pred_class.reshape(blue_crop.shape)
                    landclass = np.array(landclass, dtype = float)
                    l_class3 = landclass == 3
                    l_class3 = l_class3.astype(np.float)
                    nir_crop = nir_crop*l_class3
                    red_crop = red_crop*l_class3
                    blue_crop = blue_crop*l_class3

                    nir_crop1 = Final_Array_Cleaner(nir_crop)
                    red_crop1 = Final_Array_Cleaner(red_crop)
                    blue_crop1 = Final_Array_Cleaner(blue_crop)
                    NDVIcrop1 = IndexCalc(nir_crop1, red_crop1)
                    EVIcrop1 = IndexEVI(nir_crop1, red_crop1, blue_crop1)
                    Ave_NDVI = np.nanmean(NDVIcrop1)
                    Ave_EVI = np.nanmean(EVIcrop1)
                    if not math.isnan(Ave_NDVI):
                        WriteTofile(dateblue, shapefile_path, Ave_NDVI, Ave_EVI)
                        print(dateblue, Ave_NDVI)


    #                     classes = ['Clouds','Crops','Water']
    #                     length = len(classes)
    #                     fig = plt.figure(figsize=(15,15))
    #                     cmap = plt.cm.get_cmap("Set1", length)
    #                     plt.imshow(landclass, cmap=cmap)
    #                     ticks = np.arange(1, length + 1)
    #                     colorbar = plt.colorbar(ticks = ticks, orientation="horizontal")
    #                     colorbar.set_ticklabels(classes)
    #                     plt.savefig('Syngenta_2018/ClassifiedImage.png')
    #                     plt.close(fig )

                        fig = plt.figure(figsize = (20, 15))
                        plt.rc('xtick', labelsize = 20)
                        plt.rc('ytick', labelsize = 20)
                        plt.subplot(221)
                        plt.title('Landsat NDVI\n', fontsize = 20 , fontweight = 'bold')
                        plt.imshow(NDVIcrop1, cmap = plt.cm.nipy_spectral)
                        plt.subplot(222)
                        plt.title('True Color Image\n', fontsize = 20, fontweight = 'bold')
                        plt.imshow(better_rgb)
                        plt.savefig(shapefile_path[0][:-4]+ '_' + dateblue +  '.png')
                        plt.close(fig)
            except:
                pass


            dateblue = 8
            dategreen = 6
            datered = 98
            datenir = 90

20161119 0.16491645348798634


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: R

20170412 0.3274482590285143
20170428 0.21261738502287508


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice


20161119 0.16916913208130313


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: R

20170311 0.6884387576307264
20170327 0.4850874588141406
20170412 0.32288775344791976
20170428 0.21513002364066192


/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/home/emmanuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: Mean of empty slice


In [ ]:
                                  #PLANETSCOPE IMAGE ANALYSIS
for subdir, dirs, files in os.walk(rasters_path):
    for file in files:
        if file.endswith('RS.tif'):
            file1 = file
            filename = os.path.join(subdir, file1)
            try:
                date =  file1[0:4] + file1[4:6] + file1[6:8] 
                raster = rasterio.open(os.path.join(subdir, file1))
                blue, green, red, nir = raster.read()
                shapefile = shapefile.to_crs(crs = raster.crs.data)
                extent_geojson = mapping(shapefile['geometry'][0])

                raster_crop, crop_affine = mask(raster, shapes = [extent_geojson], crop = True)
                raster_meta = raster.meta.copy()
                raster_meta.update({'driver': 'GTiff', 'height':raster_crop.shape[1],
                                    'width':raster_crop.shape[2], 'transform':crop_affine})
                cr_ext = rasterio.transform.array_bounds(raster_meta['height'],
                                                         raster_meta['width'], raster_meta['transform'])
                bound_order = [0, 1, 2, 3]
                cr_extent = [cr_ext[b] for b in bound_order]
                bluecrop = raster_crop[0]*0.0001
                greencrop = raster_crop[1]*0.0001
                redcrop = raster_crop[2]*0.0001
                nircrop = raster_crop[3]*0.0001
                NDVIcrop = IndexCalc(nircrop, redcrop)
                NDWIcrop = IndexCalc(greencrop, nircrop)
                if np.nanmean(bluecrop) != 0.0 and np.nanmean(NDVIcrop) != 0.0:
                    rgb = np.dstack((redcrop, greencrop, bluecrop))
                    better_rgb = exposure.rescale_intensity(rgb, in_range=(0, 0.25))

                    cdf = allPixels1(bluecrop, greencrop, redcrop, nircrop, NDVIcrop, NDWIcrop)
                    img = cdf.iloc[:,:].values
                    pred_class = PlanetscopeClassifier.predict(img)
                    landclass = pred_class.reshape(bluecrop.shape)
                    landclass = np.array(landclass, dtype = float)
                    l_class3 = landclass == 2
                    l_class3 = l_class3.astype(np.float)
                    nir_crop = nircrop*l_class3
                    red_crop = redcrop*l_class3
                    blue_crop = bluecrop*l_class3

                    nir_crop1 = Final_Array_Cleaner(nir_crop)
                    red_crop1 = Final_Array_Cleaner(red_crop)
                    blue_crop1 = Final_Array_Cleaner(blue_crop)
                    NDVIcrop1 = IndexCalc(nir_crop1, red_crop1)
                    EVIcrop1 = IndexEVI(nir_crop1, red_crop1, blue_crop1)
                    Ave_NDVI = np.nanmean(NDVIcrop1)
                    Ave_EVI = np.nanmean(EVIcrop1)
                    if not math.isnan(Ave_NDVI):
                        WriteTofile(date, shapefile_path, Ave_NDVI, Ave_EVI)
                        print(date, Ave_EVI)



#                         classes = ['No data','Data']
#                         length = len(classes)
#                         fig = plt.figure(figsize=(15,15))
#                         plt.rc('xtick', labelsize = 25)
#                         plt.rc('ytick', labelsize = 25)
#                         cmap = plt.cm.get_cmap("Set1", length)
#                         plt.title('Pixel Classification\n', fontsize = 30 , fontweight = 'bold')
#                         plt.imshow(landclass, cmap=cmap)
#                         ticks = np.arange(1, length + 1)
#                         colorbar = plt.colorbar(ticks = ticks, orientation="horizontal")
#                         colorbar.set_ticklabels(classes)
#                         plt.savefig(shapefile_path[0][:-4] +'Classified'+ '_' + date +  '.png')
#                         plt.close(fig )

                        fig = plt.figure(figsize = (20, 20))
                        plt.rc('xtick', labelsize = 25)
                        plt.rc('ytick', labelsize = 25)
                        plt.subplot(221)
                        plt.title('EVI Image\n', fontsize = 30 , fontweight = 'bold')
                        plt.imshow(EVIcrop1, cmap = plt.cm.nipy_spectral)
                        #plt.colorbar(orientation = 'vertical', shrink = 0.7)
                        plt.subplot(222)
                        plt.title('True colour image\n', fontsize = 30, fontweight = 'bold')
                        plt.imshow(better_rgb)
                        plt.savefig(shapefile_path[0][:-4]+ '_' + date +  '.png')
                        plt.close(fig)
            except:
                pass